In [34]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools, ftfy
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')


def retry_post(resource, r_post):
    r_post_old = r_post
    if r_post:
        r_post_retry = api.retry_post(resource,r_post)
        return r_post_old, r_post_retry
    return r_post_old

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [ ]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

### Andamento Processo (Sentença)

### Personagem

#### carrega dados codigo e nome do personagem especial (jurídica) e personagens pessoa fĩsicaa

#### carrega dados dos advogados

#### divide as associações aos processos entre personagens (fisico ou juridico) e advogados

### Documentos (Iniciais e Contestação)

### Arquivos similaridade

In [39]:
a = api.get('processossimilaresreport', '2205382')
a

<Response [200]>

In [40]:
api.response_to_json(a)

{'processo_base': {'processo_tj': '2016.204.024263-0',
  'processo_cnj': '0024418-11.2016.8.19.0204',
  'comarca': 'Regional de Bangu',
  'serventia': 'Cartório do 29º Juizado Especial Cível',
  'assunto': 'Indenização Por Dano Moral - Outros',
  'classe': 'Procedimento do Juizado Especial Cível',
  'competencia': 'Juizado Especial Cível',
  'advogados': [],
  'personagens': [{'id': 515863,
    'processo_tj': '2016.204.024263-0',
    'processo_cnj': '0024418-11.2016.8.19.0204',
    'nome_personagem': 'ITAÚ UNIBANCO S/A',
    'tipo_personagem': 'Réu',
    'participacao': 'P'},
   {'id': 503721,
    'processo_tj': '2016.204.024263-0',
    'processo_cnj': '0024418-11.2016.8.19.0204',
    'nome_personagem': 'ISLANE BOMFIM DA SILVA',
    'tipo_personagem': 'Autor',
    'participacao': 'A'}],
  'documentos': [],
  'sentencas': [{'id': 796847,
    'processo_tj': '2016.204.024263-0',
    'processo_cnj': '0024418-11.2016.8.19.0204',
    'texto_sentenca': 'Homologo o projeto de sentença apresent

In [ ]:
{'processo_base': {'processo_tj': '2015.205.043963-7',
  'processo_cnj': '0044394-35.2015.8.19.0205',
  'comarca': 'Regional de Campo Grande',
  'serventia': 'Cartório do 18º Juizado Especial Cível',
  'assunto': 'Cobrança de Quantia Indevida',
  'classe': 'Procedimento do Juizado Especial Cível',
  'competencia': 'Juizado Especial Cível',
  'advogados': [{'id': 612776,
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'nome': 'FABIA MAMEDE SOUZA DA SILVA',
    'oab': 'RJ113302',
    'polo': None},
   {'id': 597714,
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'nome': 'ALEXANDRE MARQUES SILVEIRA',
    'oab': 'RJ146575',
    'polo': None},
   {'id': 613101,
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'nome': 'MARTA THEODORO BORGES',
    'oab': 'RJ143786',
    'polo': 'A'}],
  'personagens': [{'id': 525265,
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'nome_personagem': 'BANCO ITAUCARD S/A',
    'tipo_personagem': 'Réu',
    'participacao': 'P'},
   {'id': 525153,
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'nome_personagem': 'CLAUDIO AUGUSTO DO NASCIMENTO MELLO',
    'tipo_personagem': 'Autor',
    'participacao': 'A'}],
  'documentos': [{'cod_documento': '0004C2E0D386E0F4DA70EBF60BE40C80F0E9C5042A223351',
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'id_tipo_documento': '14',
    'tipo_documento': 'Petição Inicial/Doc. de Origem'},
   {'cod_documento': '0004325EAECBA5D6CCC670FBF913644C3877C5042E112B06',
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'id_tipo_documento': '74',
    'tipo_documento': 'Contestação'}],
  'sentencas': [{'id': 776655,
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'texto_sentenca': 'Processo nº: 0044394-35.2015.8.19.0205\nAutora: CLAUDIO AUGUSTO DO NASCIMENTO MELLO \nRéu: BANCO ITAUCARD S/A\n\nPROJETO DE SENTENÇA\nDispensado o relatório, nos termos do artigo 38 da Lei 9.099/95.\nA parte Autora informa que é cliente da ré na prestação do serviço de financiamento de veículo. Informa que foram aplicados ao valor financiado juros exorbitantes e ilegais. Objetiva assim: 1) a revisão contratual com a devolução dos valores pagos indevidamente; 2) a reparação pelos danos morais sofridos.\nContudo, através de exame dos autos, pode se depreender que o autor vem a juízo questionar a cobrança de juros, que entende abusivos e fora do patamar aceitável, proporcionando o desequilíbrio contratual. A produção da prova pericial contábil revela-se imprescindível à formação do juízo de certeza necessário a prolação de uma sentença de mérito, demonstrando assim a complexidade da causa e a decorrente incompetência desse juízo.\nAnte o exposto, julgo extinto o processo, sem resolução do mérito, com fulcro no art. 51, II da lei 9099/95.\nSem custas nem honorários advocatícios, nos termos do art. 55 da Lei n. 9.099/95.\nObserve o Cartório que as publicações para o Réu deverão ser feitas conforme requerido nos autos. \n\n\nSubmeto o presente projeto de sentença à apreciação do Juiz Togado, de acordo com o art. 40 da Lei n. 9.099/95.\nRio de Janeiro, 12 de fevereiro de 2016.\n\nMARCELO DE OLIVEIRA HEMERLY.\nJuiz Leigo',
    'ato_juiz': 'Extinto o processo por desistência',
    'nome_juiz': 'Marcelo de Oliveira Hemerly',
    'matricula_juiz': '116303',
    'cargo_juiz': 'Juiz Leigo'},
   {'id': 786232,
    'processo_tj': '2015.205.043963-7',
    'processo_cnj': '0044394-35.2015.8.19.0205',
    'texto_sentenca': 'Homologo, por sentença, para que surta seus efeitos legais, o projeto de sentença elaborado pelo Juiz Leigo, na forma do artigo 40 da Lei .º 9.099/95. \n\nApós o trânsito em julgado, tratando-se de sentença de improcedência ou de extinção do feito sem resolução do mérito, dê-se baixa e arquivem-se imediatamente. \n\nApós o trânsito em julgado e comprovado eventual depósito, fica autorizada a expedição de mandado de pagamento. \n\nCientes as partes do disposto no art. 52, IV, da Lei 9.099/95, quanto à necessidade de cumprimento voluntário da sentença, sob pena de penhora, dispensada nova citação. Ficam, ainda, intimadas as partes de que, nas sentenças que fixarem obrigação de pagar, caso o devedor não pague a quantia certa a que foi condenado, no prazo de 15 dias, contados do trânsito em julgado da sentença ou do acórdão, o valor da condenação será acrescido da multa de 10%, prevista no art. 475-J do CPC, independente de nova intimação. \n\t\nCertificado o trânsito em julgado e nada mais se requerendo, no prazo de 15 dias, dê-se baixa e arquive-se, advertidas as partes da possibilidade de se incinerarem os autos após 90 dias do arquivamento definitivo (art. 1º, Ato Normativo Conjunto 01/2005, publicado no Diário Oficial em 07 de janeiro de 2005), pelo que defiro, desde logo, o desentranhamento dos documentos originais, por quem os juntou, mediante a substituição por cópias.',
    'ato_juiz': 'Extinta a punib. por cumprimento da susp. cond. do processo',
    'nome_juiz': 'Anelise de Faria Martorell Duarte',
    'matricula_juiz': '28811',
    'cargo_juiz': 'Juiz'}],
  'estatistica': [{'referencia': 17, 'referenciado': 21}],
  'processos_similares': [{'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1856445/',
    'similaridade': 57.12,
    'processo_similar_tj': '2016.205.011663-2',
    'processo_similar_cnj': '0011681-70.2016.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1852563/',
    'similaridade': 56.12,
    'processo_similar_tj': '2016.204.003606-8',
    'processo_similar_cnj': '0003614-22.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2174849/',
    'similaridade': 55.71,
    'processo_similar_tj': '2015.205.039787-4',
    'processo_similar_cnj': '0040188-75.2015.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2033375/',
    'similaridade': 55.05,
    'processo_similar_tj': '2015.204.016861-0',
    'processo_similar_cnj': '0016952-97.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1997747/',
    'similaridade': 54.26,
    'processo_similar_tj': '2015.204.003443-4',
    'processo_similar_cnj': '0003451-76.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1927631/',
    'similaridade': 52.36,
    'processo_similar_tj': '2015.204.009732-8',
    'processo_similar_cnj': '0009782-74.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2070339/',
    'similaridade': 52.01,
    'processo_similar_tj': '2016.204.024263-0',
    'processo_similar_cnj': '0024418-11.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1590351/',
    'similaridade': 49.03,
    'processo_similar_tj': '2015.204.021545-3',
    'processo_similar_cnj': '0021665-18.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2269626/',
    'similaridade': 48.42,
    'processo_similar_tj': '2015.204.027082-8',
    'processo_similar_cnj': '0027225-38.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1791871/',
    'similaridade': 48.34,
    'processo_similar_tj': '2015.204.005976-5',
    'processo_similar_cnj': '0006001-44.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1894411/',
    'similaridade': 48.11,
    'processo_similar_tj': '2016.204.026825-3',
    'processo_similar_cnj': '0027019-87.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1823074/',
    'similaridade': 47.87,
    'processo_similar_tj': '2016.205.020587-2',
    'processo_similar_cnj': '0020745-07.2016.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1624333/',
    'similaridade': 47.78,
    'processo_similar_tj': '2015.204.022950-6',
    'processo_similar_cnj': '0023073-44.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2209999/',
    'similaridade': 47.77,
    'processo_similar_tj': '2015.205.045415-8',
    'processo_similar_cnj': '0045854-57.2015.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1964460/',
    'similaridade': 47.75,
    'processo_similar_tj': '2015.205.036148-0',
    'processo_similar_cnj': '0036499-23.2015.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1929325/',
    'similaridade': 47.42,
    'processo_similar_tj': '2016.204.023769-4',
    'processo_similar_cnj': '0023922-79.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1966048/',
    'similaridade': 47.29,
    'processo_similar_tj': '2015.205.049755-8',
    'processo_similar_cnj': '0050209-13.2015.8.19.0205'}]},
 'processo_similar': {'processo_tj': '2016.205.011663-2',
  'processo_cnj': '0011681-70.2016.8.19.0205',
  'comarca': 'Regional de Campo Grande',
  'serventia': 'Cartório do 18º Juizado Especial Cível',
  'assunto': 'Indenização Por Dano Moral - Outras',
  'classe': 'Procedimento do Juizado Especial Cível',
  'competencia': 'Juizado Especial Cível',
  'advogados': [{'id': 697106,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome': 'MAURÍCIO COIMBRA GUILHERME FERREIRA',
    'oab': 'RJ151056',
    'polo': 'P'},
   {'id': 702798,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome': 'SUELY DOS SANTOS FONTOURA',
    'oab': 'RJ052594',
    'polo': 'A'},
   {'id': 733812,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome': 'CANDIDA RICARDO DE PAULA',
    'oab': 'RJ128104',
    'polo': 'P'},
   {'id': 704284,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome': 'VILIA MARIA ASSUNÇAO SANTOS',
    'oab': 'RJ106099',
    'polo': 'A'},
   {'id': 704656,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome': 'GILBERTO DE FREITAS MAGALHÃES JUNIOR',
    'oab': 'RJ123792',
    'polo': 'P'}],
  'personagens': [{'id': 634705,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome_personagem': 'BANCO ITAÚ S/A',
    'tipo_personagem': 'Réu',
    'participacao': 'P'},
   {'id': 652547,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome_personagem': 'BANCO ITAUCARD S/A',
    'tipo_personagem': 'Réu',
    'participacao': 'P'},
   {'id': 616195,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome_personagem': 'VÍLIA MARIA ASSUNÇÃO SANTOS',
    'tipo_personagem': 'Autor',
    'participacao': 'A'},
   {'id': 616783,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'nome_personagem': 'MAURO TRINDADE DA SILVA COSTA',
    'tipo_personagem': 'Autor',
    'participacao': 'A'}],
  'documentos': [],
  'sentencas': [{'id': 767865,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'texto_sentenca': 'Homologo, por sentença, para que surta seus efeitos legais, o projeto de sentença elaborado pelo Juiz Leigo, na forma do artigo 40 da Lei .º 9.099/95. \n\nApós o trânsito em julgado, tratando-se de sentença de improcedência ou de extinção do feito sem resolução do mérito, dê-se baixa e arquivem-se imediatamente. \n\nApós o trânsito em julgado e comprovado eventual depósito, fica autorizada a expedição de mandado de pagamento. \n\nCientes as partes do disposto no art. 52, IV, da Lei 9.099/95, quanto à necessidade de cumprimento voluntário da sentença, sob pena de penhora, dispensada nova citação. Ficam, ainda, intimadas as partes de que, nas sentenças que fixarem obrigação de pagar, caso o devedor não pague a quantia certa a que foi condenado, no prazo de 15 dias, contados do trânsito em julgado da sentença ou do acórdão, o valor da condenação será acrescido da multa de 10%, prevista no art. 523, § 1º do CPC, independente de nova intimação. \n\t\nCertificado o trânsito em julgado e nada mais se requerendo, no prazo de 15 dias, dê-se baixa e arquive-se, advertidas as partes da possibilidade de se incinerarem os autos após 90 dias do arquivamento definitivo (art. 1º, Ato Normativo Conjunto 01/2005, publicado no Diário Oficial em 07 de janeiro de 2005), pelo que defiro, desde logo, o desentranhamento dos documentos originais, por quem os juntou, mediante a substituição por cópias.',
    'ato_juiz': 'Extinta a punib. por cumprimento da susp. cond. do processo',
    'nome_juiz': 'Anelise de Faria Martorell Duarte',
    'matricula_juiz': '28811',
    'cargo_juiz': 'Juiz'},
   {'id': 760132,
    'processo_tj': '2016.205.011663-2',
    'processo_cnj': '0011681-70.2016.8.19.0205',
    'texto_sentenca': 'Processo nº: 0011681-70.2016.8.19.0205\nAutora: MAURO TRINDADE DA SILVA COSTA \n1º Réu: BANCO ITAUCARD S/A\n2º Réu: BANCO ITAÚ S/A\n\nPROJETO DE SENTENÇA\nDispensado o relatório, nos termos do artigo 38 da Lei 9.099/95.\nA parte Autora informa que é cliente da ré na prestação dos serviços de cartão de crédito prestados pelos réus. Informa que, por passar por situação financeira delicada, não pôde pagar a fatura com vencimento em março deste ano. Contudo, entrou em contato com os réus para tentar o parcelamento dos valores de debito. Afirma que os prepostos dos réus inseriram juros absurdos e encargos que impossibilitam aceitar o parcelamento. Objetiva assim: 1) a determinação para que os réus não debitem da sua conta os valores correspondentes ao mínimo do pagamento da fatura; 2) que os réus se abstenham de incluir seu nome nos cadastros de inadimplentes; 3) que sejam aplicados ao parcelamento os juros legais; 4) a reparação pelos danos morais sofridos.\nEntretanto, através de exame dos autos, pode se depreender que o autor vem a juízo questionar a cobrança de juros, que entende abusivos e fora do patamar aceitável. Assim, a produção da prova pericial contábil revela-se imprescindível à formação do juízo de certeza necessário a prolação de uma sentença de mérito, demonstrando assim a complexidade da causa e a decorrente incompetência desse juízo.\nAnte o exposto, julgo extinto o processo, sem resolução do mérito, com fulcro no art. 51, II da lei 9099/95.\nSem custas nem honorários advocatícios, nos termos do art. 55 da Lei n. 9.099/95.\nObserve o Cartório que as publicações para o Réu deverão ser feitas conforme requerido nos autos. \nSubmeto o presente projeto de sentença à apreciação do Juiz Togado, de acordo com o art. 40 da Lei n. 9.099/95.\nRio de Janeiro, 10 de junho de 2016.\n\nMARCELO DE OLIVEIRA HEMERLY\nJuiz Leigo',
    'ato_juiz': 'Extinto o processo por desistência',
    'nome_juiz': 'Marcelo de Oliveira Hemerly',
    'matricula_juiz': '116303',
    'cargo_juiz': 'Juiz Leigo'}],
  'estatistica': [{'referencia': 16, 'referenciado': 16}],
  'processos_similares': [{'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1787584/',
    'similaridade': 73.07,
    'processo_similar_tj': '2016.204.003606-8',
    'processo_similar_cnj': '0003614-22.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1911820/',
    'similaridade': 66.92,
    'processo_similar_tj': '2015.204.003443-4',
    'processo_similar_cnj': '0003451-76.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1888139/',
    'similaridade': 62.96,
    'processo_similar_tj': '2015.204.027082-8',
    'processo_similar_cnj': '0027225-38.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2214764/',
    'similaridade': 61.81,
    'processo_similar_tj': '2015.205.039787-4',
    'processo_similar_cnj': '0040188-75.2015.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2066857/',
    'similaridade': 59.0,
    'processo_similar_tj': '2016.204.024263-0',
    'processo_similar_cnj': '0024418-11.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1619243/',
    'similaridade': 58.66,
    'processo_similar_tj': '2015.205.045415-8',
    'processo_similar_cnj': '0045854-57.2015.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1886159/',
    'similaridade': 57.12,
    'processo_similar_tj': '2015.205.043963-7',
    'processo_similar_cnj': '0044394-35.2015.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1961679/',
    'similaridade': 55.31,
    'processo_similar_tj': '2015.204.016861-0',
    'processo_similar_cnj': '0016952-97.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2286062/',
    'similaridade': 54.66,
    'processo_similar_tj': '2016.204.001648-3',
    'processo_similar_cnj': '0001650-91.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2106926/',
    'similaridade': 50.18,
    'processo_similar_tj': '2016.204.006347-3',
    'processo_similar_cnj': '0006363-12.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1623447/',
    'similaridade': 49.93,
    'processo_similar_tj': '2015.204.025951-1',
    'processo_similar_cnj': '0026089-06.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2177604/',
    'similaridade': 49.72,
    'processo_similar_tj': '2015.205.052746-0',
    'processo_similar_cnj': '0053198-89.2015.8.19.0205'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1625344/',
    'similaridade': 48.83,
    'processo_similar_tj': '2015.204.010378-0',
    'processo_similar_cnj': '0010427-02.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1695624/',
    'similaridade': 48.33,
    'processo_similar_tj': '2015.204.030606-9',
    'processo_similar_cnj': '0030770-19.2015.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/1590614/',
    'similaridade': 47.96,
    'processo_similar_tj': '2016.204.003811-9',
    'processo_similar_cnj': '0003815-14.2016.8.19.0204'},
   {'id': 'http://0.0.0.0:9000/api/models/processossimilaresreport/2001129/',
    'similaridade': 47.84,
    'processo_similar_tj': '2016.204.013056-5',
    'processo_similar_cnj': '0013087-32.2016.8.19.0204'}]}}